# Retrieval-Augmented Generation (RAG) com LangChian e Mistral.

---

Os LLMs conseguem raciocinar sobre uma variedade de tópicos, mas seu conhecimento é restrito aos dados públicos disponíveis até a data em que foram treinados. Para desenvolver aplicativos de IA que possam lidar com dados privados ou informações adicionadas após o limite de treinamento, é essencial complementar o conhecimento do modelo com as informações específicas necessárias. Esse processo de buscar as informações relevantes e incorporá-las no prompt do modelo é chamado de Geração Aumentada por Recuperação (RAG).

In [7]:
pip install langchain_mistralai langchain-community langchainhub langchain-chroma bs4

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 332.8/332.8 kB 33.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 975.5/975.5 kB 50.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.4/127.4 kB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 559.5/559.5 kB 37.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 46.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 65.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.9/59.9 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━

In [9]:
import bs4
from langchain import hub
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_chroma import Chroma
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.prompts import ChatPromptTemplate
from langchain_mistralai import MistralAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_mistralai import ChatMistralAI
import getpass
import os

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass()
os.environ["MISTRAL_API_KEY"] = getpass.getpass()
os.environ["HF_API_KEY"] = getpass.getpass()

··········
··········
··········


### Carregar, dividir e indexar o conteúdo para criar um mecanismo de recuperação.

Primeiramente, é necessário carregar os dados. Neste notebook, vamos extrair as perguntas e respostas mais frequentes sobre os programas oferecidos pela Secretaria de Estado de Desenvolvimento Social.

In [ ]:
#Utilizando WebBaseLoader para carregar conteúdo de páginas da web.
loader = WebBaseLoader(
    web_paths=([
        "https://goias.gov.br/social/perguntas-e-respostas-frequentes-maes-de-goias/",
        "https://goias.gov.br/social/perguntas-e-respostas-frequentes-goias-por-elas/",
        "https://goias.gov.br/social/perguntas-e-respostas-frequentes-dignidade/",
        "https://goias.gov.br/social/perguntas-e-respostas-frequentes-passaporte-intermunicipal-da-pessoa-idosa/",
        "https://goias.gov.br/social/perguntas-e-respostas-frequentes-passe-livre-estudantil/",
        "https://goias.gov.br/social/perguntas-e-respostas-frequentes-aprendiz-do-futuro/",
        "https://goias.gov.br/social/perguntas-e-respostas-frequentes-carteira-de-identificacao-do-autista2/",

    ]), # Especifica a URL de onde o conteúdo será carregado
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("platform-content", "platform-content")
        )
    ),
)
docs = loader.load() #Carrega o conteúdo da URL especificada e aplica o filtro definido.

In [ ]:
print(len(docs[0].page_content))
print(len(docs[1].page_content))
print(len(docs[2].page_content))
print(len(docs[3].page_content))
print(len(docs[4].page_content))
print(len(docs[5].page_content))
print(len(docs[6].page_content))


1893
1399
1003
1217
2094
1545
1472


Dividir um documento em pedaços menores ajuda a processar e analisar textos longos de maneira mais eficiente, especialmente em tarefas como indexação e recuperação de informações.

Em casos que a quantidade de informações salvas em um documento é muito extensa, precisamos realizar a divisão desse documento.

A função acima mostra a quantidade de caracteres que cada pagina tem, como são valores pequenos por cada pagina, vamos realizar um dicisão só para demonstração, visto que não seria tão necessário neste cenário.

In [ ]:
divisor = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200, add_start_index=True)
documento_dividido = divisor.split_documents(docs)

Agora precisamos indexar nossos pedaços de texto para que possamos pesquisar sobre eles durante a execução. A maneira mais comum de fazer isso é incorporar o conteúdo de cada divisão do documento e inserir essas incorporações em um banco de dados de vetores (ou armazenamento de vetores)

In [ ]:
vectorstore = Chroma.from_documents(documents=documento_dividido, embedding=MistralAIEmbeddings()) # cria um vetor de armazenamento a partir dos pedaços do documento usando embeddings do Mistral
retriever = vectorstore.as_retriever() # é configurado para recuperar documentos do vetor de armazenamento.

Crie um system_prompt que define o contexto e instruções para o assistente responder às perguntas.

In [ ]:
# 2. Incorpore o recuperador em uma cadeia de perguntas e respostas.
system_prompt = (
    "Você é um assistente para tarefas de resposta a perguntas. "
    "Use as seguintes partes do contexto recuperado para responder"
    "a questão. Se você não sabe a resposta, diga que você"
    "não sei. Use no máximo três frases e mantenha a"
    "resposta de forma concisa."
    "\n\n"
    "{context}"
)

ChatPromptTemplate.from_messages cria um template de prompt com mensagens para o sistema e o humano. O sistema recebe system_prompt e o humano recebe a entrada do usuário ({input}).

In [ ]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

In [ ]:
llm = ChatMistralAI(model="mistral-large-latest")
#Esta função cria uma cadeia que pode processar documentos para responder perguntas. Ela combina o modelo de linguagem (llm) com um prompt específico.
question_answer_chain = create_stuff_documents_chain(llm, prompt)
#Esta função cria uma cadeia que primeiro recupera documentos relevantes e depois usa outra cadeia para processá-los e responder perguntas.
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [ ]:
response = rag_chain.invoke({"input": "Quem pode participar do aprendiz do futuro?"})
response["answer"]

'O Programa Aprendiz do Futuro está aberto para jovens de 14 até 15 anos e 11 meses, com prioridade para aqueles em situação de vulnerabilidade econômico-social, estudantes da rede pública ou bolsistas 100% de rede particular. Além disso, o programa prioriza jovens membros de etnias tradicionais e famílias com mulheres vítimas de violência doméstica. Também há cotas de 5 a 10% para pessoas com deficiência, sem limite de idade.'